In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install tensorflow

In [ ]:
!pip install TensorBoard

In [ ]:
!pip install keras

In [ ]:
import matplotlib.pyplot

In [2]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import shutil
import zipfile
from google.colab import files

# Step 2: Prepare the Data (Example: Using PASCAL VOC dataset)
def load_pascal_voc_dataset(path: str ="/content/drive/MyDrive/pascal_dataset"):
    images = []
    labels = []

    # Iterate over all XML annotation files in the dataset
    for annotation_file in os.listdir(os.path.join("/content/drive/MyDrive/pascal_dataset", 'Annotations')):
        annotation_path = os.path.join("/content/drive/MyDrive/pascal_dataset", 'Annotations', annotation_file)

        # Parse XML annotation file
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        # Extract bounding box coordinates and class label
        xmin = int(root.find('.//xmin').text)
        ymin = int(root.find('.//ymin').text)
        xmax = int(root.find('.//xmax').text)
        ymax = int(root.find('.//ymax').text)
        label = root.find('.//name').text

        # Read corresponding image and preprocess
        image_file = os.path.join("/content/drive/MyDrive/pascal_dataset", 'JPEGImages', annotation_file.replace('.xml', '.jpg'))
        image = cv2.imread(image_file, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (640, 640))  # Resize images to the desired input shape

        # Append to the dataset
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

# Replace 'path_to_pascal_dataset' with the actual path to your PASCAL VOC dataset
pascal_voc_dataset_path = "/content/drive/MyDrive/pascal_dataset"
#X, y = load_pascal_voc_dataset(pascal_voc_dataset_path)

# Define your class names
class_names = ['penis', 'vagina', 'breast']

# Load dataset using specified class names
X, y = load_pascal_voc_dataset(pascal_voc_dataset_path)

# Normalize pixel values to the range [0, 1]
X = X / 255.0

# Encode class labels (convert string labels to numerical representation)
label_encoder = {label: i for i, label in enumerate(np.unique(y))}
y_encoded = np.array([label_encoder[label] for label in y])

# Split data into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)


# Step 1: Choose a Model Architecture (Example: Dense Neural Network)
model = models.Sequential([
    layers.Flatten(input_shape=(640, 640, 3)),  # Input layer for image data
    layers.Dense(128, activation='relu'),   # Hidden layer with 128 neurons and ReLU activation
    layers.Dropout(0.2),                    # Dropout layer to prevent overfitting
    #layers.Dense(3, activation='softmax')
    layers.Dense(len(label_encoder))        # Output layer with the number of neurons equal to the number of classes
])

# Step 3: Compile the Model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# # Define TensorBoard callback
# tensorboard_callback = cb.TensorBoard(log_dir='./logs', histogram_freq=1)


# Step 4: Train the Model
model.fit(x_train, y_train, epochs=200, validation_data=(x_val, y_val))

# Step 5: Evaluate the Model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")

# Step 6: Save the TensorFlow Model in SavedModel format
# Save the TensorFlow Model in SavedModel format
model.save('my_model', save_format='h5')
# model.save('my_model.keras')

# # Load TensorBoard in Colab for visualization
# %load_ext tensorboard
# %tensorboard --logdir=./logs


model_filename = 'my_model'
zip_filename = 'model.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(model_filename, os.path.basename(model_filename))

drive_folder = '/content/drive/MyDrive'  # Change Your_Folder to the desired folder path in Google Drive

shutil.move(zip_filename, os.path.join(drive_folder, zip_filename))



Epoch 1/200
6/6 [==============================] - 3s 249ms/step - loss: 648.7349 - accuracy: 0.4720 - val_loss: 719.3860 - val_accuracy: 0.6333
Epoch 2/200
6/6 [==============================] - 1s 169ms/step - loss: 405.2440 - accuracy: 0.6149 - val_loss: 380.4190 - val_accuracy: 0.2667
Epoch 3/200
6/6 [==============================] - 1s 168ms/step - loss: 190.0901 - accuracy: 0.4907 - val_loss: 234.7508 - val_accuracy: 0.6333
Epoch 4/200
6/6 [==============================] - 1s 187ms/step - loss: 161.8561 - accuracy: 0.5280 - val_loss: 148.5024 - val_accuracy: 0.2667
Epoch 5/200
6/6 [==============================] - 1s 213ms/step - loss: 150.6706 - accuracy: 0.5093 - val_loss: 157.1681 - val_accuracy: 0.6333
Epoch 6/200
6/6 [==============================] - 1s 210ms/step - loss: 122.5644 - accuracy: 0.5342 - val_loss: 64.7478 - val_accuracy: 0.6667
Epoch 7/200
6/6 [==============================] - 1s 213ms/step - loss: 43.4870 - accuracy: 0.6149 - val_loss: 18.1973 - val_accur

'/content/drive/MyDrive/model.zip'

In [8]:
import zipfile
import os
import shutil
from google.colab import files

model_filename = 'my_model'
zip_filename = 'model.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write(model_filename, os.path.basename(model_filename))

drive_folder = '/content/drive/MyDrive'  # Change Your_Folder to the desired folder path in Google Drive

shutil.move(zip_filename, os.path.join(drive_folder, zip_filename))

files.download(zip_filename)


FileNotFoundError: Cannot find file: model.zip

In [ ]:
# # This code is for grayscale

# import os
# import cv2
# import xml.etree.ElementTree as ET
# import numpy as np
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Step 2: Prepare the Data (Example: Using PASCAL VOC dataset)
# def load_pascal_voc_dataset(path: str ="/content/drive/MyDrive/pascal_dataset"):
#     images = []
#     labels = []

#     # Iterate over all XML annotation files in the dataset
#     for annotation_file in os.listdir(os.path.join("/content/drive/MyDrive/pascal_dataset", 'Annotations')):
#         annotation_path = os.path.join("/content/drive/MyDrive/pascal_dataset", 'Annotations', annotation_file)

#         # Parse XML annotation file
#         tree = ET.parse(annotation_path)
#         root = tree.getroot()

#         # Extract bounding box coordinates and class label
#         xmin = int(root.find('.//xmin').text)
#         ymin = int(root.find('.//ymin').text)
#         xmax = int(root.find('.//xmax').text)
#         ymax = int(root.find('.//ymax').text)
#         label = root.find('.//name').text

#         # Read corresponding image and preprocess
#         image_file = os.path.join("/content/drive/MyDrive/pascal_dataset", 'JPEGImages', annotation_file.replace('.xml', '.jpg'))
#         image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
#         image = cv2.resize(image, (640, 640))  # Resize images to the desired input shape

#         # Append to the dataset
#         images.append(image)
#         labels.append(label)

#     return np.array(images), np.array(labels)

# # Replace 'path_to_pascal_dataset' with the actual path to your PASCAL VOC dataset
# pascal_voc_dataset_path = "/content/drive/MyDrive/pascal_dataset"
# X, y = load_pascal_voc_dataset(pascal_voc_dataset_path)

# # Normalize pixel values to the range [0, 1]
# X = X / 255.0

# # Encode class labels (convert string labels to numerical representation)
# label_encoder = {label: i for i, label in enumerate(np.unique(y))}
# y_encoded = np.array([label_encoder[label] for label in y])

# # Split data into training, validation, and test sets
# x_train, x_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42)
# x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# # Step 1: Choose a Model Architecture (Example: Dense Neural Network)
# model = models.Sequential([
#     layers.Flatten(input_shape=(640, 640, 1)),  # Input layer for image data
#     layers.Dense(128, activation='relu'),   # Hidden layer with 128 neurons and ReLU activation
#     layers.Dropout(0.2),                    # Dropout layer to prevent overfitting
#     #layers.Dense(3, activation='softmax')
#     layers.Dense(len(label_encoder))        # Output layer with the number of neurons equal to the number of classes
# ])

# # Step 3: Compile the Model
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# # Step 4: Train the Model
# model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# # Step 5: Evaluate the Model
# test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
# print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")

# # Step 6: Save the TensorFlow Model in SavedModel format
# model.save('my_model')


Epoch 1/5
5/5 [==============================] - 13s 2s/step - loss: 191.9013 - accuracy: 0.4397 - val_loss: 200.1626 - val_accuracy: 0.6667
Epoch 2/5
5/5 [==============================] - 10s 2s/step - loss: 258.9073 - accuracy: 0.6738 - val_loss: 125.0387 - val_accuracy: 0.6667
Epoch 3/5
5/5 [==============================] - 5s 1s/step - loss: 100.9757 - accuracy: 0.3972 - val_loss: 46.3122 - val_accuracy: 0.6667
Epoch 4/5
5/5 [==============================] - 5s 1s/step - loss: 93.6484 - accuracy: 0.6170 - val_loss: 51.8063 - val_accuracy: 0.3000
Epoch 5/5
5/5 [==============================] - 7s 1s/step - loss: 53.1136 - accuracy: 0.4894 - val_loss: 37.6635 - val_accuracy: 0.4333
1/1 - 0s - loss: 35.2780 - accuracy: 0.3548 - 145ms/epoch - 145ms/step

Test Accuracy: 35.48%


In [ ]:
# Provide a link to download the model in a zip file
model_path = '/content/path_to_my_model'  # Replace with the actual path
zip_file_path = '/content/model.zip'

# Zip the model folder
!zip -r {zip_file_path} {model_path}

# Download the zip file
from google.colab import files
files.download(zip_file_path)